In [1]:
%%capture
%run 03_block.ipynb

We can validate a mined block pow very easily. 

In [2]:
def val_pow(mb):
    mb_b        = bytes(mb)
    candidate   = mb_b + str(mb.nonce).encode()
    candidate_h = sha(candidate)
    if candidate_h[:mb.diff] == '0'*mb.diff: return True
    else                                   : return False
    
assert val_pow(mb)

If we change something in the block the pow should break.

In [3]:
mb.prev_hash = rh()
assert not val_pow(mb)

## Blockchain

Consists of a number of blocks. Each block points to the block that came before it. Imagine a linked list of blocks linked by `prev_hash`.

In [4]:
class Blockchain:
    def __init__(self, genesis):
        self.balances,self.nonces = {},{}
        self.blocks   = []
        self.init(genesis)
        
    def init(self, genesis):
        for tx in genesis.txs: 
            self.balances[tx.to] = tx.value
            self.nonces  [tx.to] = 0
        self.blocks.append(genesis)

    def candidate(self, txs):
        return Block(txs, self.blocks[-1].hash, len(self.blocks)+1)
        
    def add(self, mb):
        assert mb.verify_pow()
        for tx in mb.txs:   assert self.apply(tx)
        assert self.verify(mb)
        self.balances[mb.miner] += mb.reward
        self.blocks.append(mb)
        return True
        
    def apply(self, tx):
        if tx.fr in self.nonces:
            assert self.nonces[tx.fr] == tx.nonce
            self.nonces[tx.fr] += 1
        else: 
            assert tx.nonce == 0
            self.nonces[tx.fr] = 0
        assert self.balances[tx.fr] - tx.value > 0
        self.balances[tx.fr] -= tx.value
        if tx.to in self.balances:   self.balances[tx.to] += tx.value
        else:                        self.balances[tx.to]  = tx.value
        if tx.to not in self.nonces: self.nonces[tx.to]    = 0
        return True
    
    def val(self, mb): 
        assert mb.val_pow()
        if len(self.blocks) > 0: assert self.blocks[-1].hash == mb.prev_hash
        for tx in mb.txs:        assert TX.verify(tx)
        return True
    
    def __str__(self):
        return '\n'.join(f'{addr2emoji(k)} {ph(k)}\nbalance: {v} \nnonce: {self.nonces[k]}\n' for k,v in self.balances.items())